# Merge county level data with contact info

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle
from tqdm import tqdm
import math
import sys
sys.path.append('..')
import load_data
from copy import deepcopy
from functions import load_usafacts_data
import requests
from bs4 import BeautifulSoup

In [2]:
df_covid = load_usafacts_data.load_daily_data(dir_mod='../')

In [3]:
with open("../data/df_county_level_cached.pkl", 'rb') as f:
    county_df = pickle.load(f)
# county_df = load_data.load_county_level(dir_mod='..')
with open("countyFIPS_to_city.pkl", 'rb') as f:
    countyFIPS_to_city = pickle.load(f)
contact_info = pd.read_csv("all_state_numbers.csv", index_col = 0)

In [4]:
idxs_used = [] # keeps track of which row indexes in the contact_info were matched
def find_contact(entry, contact_info, idx):
    '''Searches the entry in county_df to see if it is contained in the contact info
    '''
    same_state = contact_info[contact_info["state"] == entry["StateNameAbbreviation"]]
    same_county = same_state[same_state["public_health"].apply(lambda x: entry["CountyName"] in x)]
    if same_county.shape[0] >= 1:
        idxs_used.append(idx)
        return ", ".join(same_county["phone_number"]), ", ".join(same_county["public_health"])
    elif entry["countyFIPS"] in countyFIPS_to_city:
        city = countyFIPS_to_city[entry["countyFIPS"]]
        try:
            same_city = same_state[same_state["public_health"].apply(lambda x: city in x)]
            if same_city.shape[0] >= 1:
                idxs_used.append(idx)
                return ", ".join(same_city["phone_number"]), ", ".join(same_city["public_health"])
            else:
                return None, None
        except:
            return None, None
    else:
        return None, None

In [5]:
health_dept_names = []
contact_numbers = []
for idx, county in tqdm(county_df.iterrows()):
    result = find_contact(county, contact_info, idx)
    health_dept_names.append(result[1])
    contact_numbers.append(result[0])
df = deepcopy(county_df)
df["HealthDeptName"] = health_dept_names
df["HealthDeptContact"] = contact_numbers

3114it [00:10, 299.94it/s]


In [6]:
df = pd.merge(df, df_covid, on='countyFIPS', how='outer')

In [7]:
matched_rows = ~df["HealthDeptContact"].isna()
nofips_rows = [k for k in np.arange(contact_info.shape[0]) if not k in idxs_used]

key_sort = 'tot_deaths' # tot_deaths, StateName
df_matched = df[matched_rows].sort_values(by=key_sort, ascending=False)
df_nocontact = df[~matched_rows].sort_values(by=key_sort, ascending=False)
df_nocontact["HealthDeptName"] = ""
df_nocontact["HealthDeptContact"] = ""
df_nofips = contact_info.iloc[nofips_rows].rename(columns={'public_health': 'HealthDeptName', 'phone_number': 'HealthDeptContact'})
for k in ['CountyName', 'countyFIPS']:
    df_nofips[k] = ''
df_nofips['StateName'] = df_nofips['state']

In [8]:
print('matched', df_matched.shape[0], 'nocontact', df_nocontact.shape[0],
      'nofips', df_nofips.shape[0])

matched 2334 nocontact 786 nofips 1081


#### Let's Now try and match the nofips data

In [9]:
url = "https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697"
res = requests.get(url)
html = res.text

In [10]:
soup = BeautifulSoup(html)
table = soup.find_all("table", {"class": "data"})[0]
trs = table.find_all("tr")

oracleFIPS = {
    "countyFIPS": [],
    "state": [],
    "CountyName": []
}

for tr in trs:
    x = tr.find_all("td")
    if len(x) == 3:
        fips = str.lstrip(x[0].text)
        county = str.lstrip(x[1].text)
        state = str.lstrip(x[2].text)
        
        oracleFIPS["countyFIPS"].append(fips)
        oracleFIPS["state"].append(state)
        oracleFIPS["CountyName"].append(county)
        
oracleFIPS = pd.DataFrame(oracleFIPS)

In [11]:
oracleFIPS.head()

,countyFIPS,state,CountyName
0,01001,AL,Autauga
1,01003,AL,Baldwin
2,01005,AL,Barbour
3,01007,AL,Bibb
4,01009,AL,Blount


In [12]:
# Let's Extract the county name
df_nofips_tmp = df_nofips
health_names = df_nofips_tmp["HealthDeptName"]
health_names = [x.split("County")[0] for x in health_names]
health_names = [str.rstrip(x) if "Health" not in x else np.nan for x in health_names]
df_nofips_tmp["CountyName"] = health_names
df_nofips_tmp = df_nofips_tmp.drop("countyFIPS", axis=1)
df_nofips_tmp = df_nofips_tmp.merge(oracleFIPS, on=["CountyName", "state"], how="left")

# Finalize 
df_nofips = df_nofips_tmp

df_nofips_matched = df_nofips[df_nofips["countyFIPS"].notnull()]
df_nofips_unmatched = df_nofips[df_nofips["countyFIPS"].isnull()]

print("Matched %s previously nofips; %s still unmatched" % (len(df_nofips_matched), len(df_nofips_unmatched)))

Matched 733 previously nofips; 348 still unmatched


In [13]:
# We only need these columns, let's append them
df_matched = df_matched[['CountyName', 'StateName', 'countyFIPS', 'HealthDeptName', 'HealthDeptContact']]
df_nofips_matched = df_nofips_matched[['CountyName', 'StateName', 'countyFIPS', 'HealthDeptName', 'HealthDeptContact']]

df_matched = df_matched.append(df_nofips_matched)
len(df_matched)

3067

In [14]:
# Update df_nofips to be currently unmatched
df_nofips = df_nofips_unmatched

# upload to gsheets

In [15]:
import pygsheets
gc = pygsheets.authorize(service_file='../creds.json')

#open the google spreadsheet (where 'PY to Gsheet Test' is the name of my sheet)
sheet_name = 'Ventilator Demand Prediction'
sh = gc.open(sheet_name) # name of the hospital

FileNotFoundError: [Errno 2] No such file or directory: '../creds.json'

In [83]:
wks = sh[4]
wks.update_value('A1', "County-level contact information, scraped from here: https://www.naccho.org/membership/lhd-directory")
wks.update_value('A2', "Columns A-E are read-only")
wks.set_dataframe(df_matched[['CountyName', 'StateName', 'countyFIPS', 'HealthDeptName', 'HealthDeptContact']], (5, 1))

/accounts/projects/vision/.local/lib/python3.7/site-packages/pygsheets/worksheet.py:1304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].astype('unicode').replace('<NA>', nan)


In [84]:
wks = sh[5]
wks.update_value('A1', "County-level contact information for counties missing a contact #")
wks.update_value('A2', "Columns A-C are read-only")
wks.set_dataframe(df_nocontact[['CountyName', 'StateName', 'countyFIPS', 'HealthDeptName', 'HealthDeptContact']], (5, 1))

In [93]:
wks = sh[6]
wks.update_value('A1', "County-level contact information (not linked to a county)")
wks.update_value('A2', "Columns A-B are read-only")
wks.set_dataframe(df_nofips[['HealthDeptName', 'HealthDeptContact', 'StateName', 'CountyName', 'countyFIPS']], (5, 1))